# Building a Personalized Travel AI Agent with MultiOn and Mem0

Create an intelligent travel companion that remembers your preferences! This notebook demonstrates how to develop an AI travel agent using MultiOn and Mem0.
Key features:

- Browser-based AI capabilities (MultiOn)
- Long-term memory for personalization (Mem0)
- Context-aware trip planning
- Personalized travel recommendations

Learn to build an assistant that enhances the travel planning experience with each interaction.

## Step 1: Set up the environment

First, install the required libraries and set up the MultiOn and Mem0 APIs. You also need to setup API keys for both MultiOn and Mem0.

- Get your MultiOn API Key: https://app.multion.ai/api-keys
- Get your Mem0 API Key: https://app.mem0.ai/dashboard/api-keys

In [ ]:
%pip install multion mem0ai

In [81]:
from mem0 import MemoryClient
from multion.client import MultiOn

# Configuration
MULTION_API_KEY = "xxx"  # Get your MultiOn API key from https://app.multion.ai/api-keys
MEM0_API_KEY = "m0-xxx"  # Get your Mem0 API key from https://app.mem0.ai/dashboard/api-keys
USER_ID = "alex" # Your user ID (can be anything)

mem0_client = MemoryClient(api_key=MEM0_API_KEY)
multion_client = MultiOn(api_key=MULTION_API_KEY)

## Step 2: Add your own data to extract memories from it using Mem0

You can add your own data to extract memories from it using Mem0.

In [82]:
text = """
Here are the details from my travel blog about myself:

I'm a passionate traveler with a well-stamped passport. Beaches are my go-to, and I always choose Airbnbs for that local touch. Exploring new cultures and cuisines is my passion, though sushi isn't on my menu. I love discovering hidden gems and connecting with locals wherever I go. For me, travel isn't just a hobby—it's a lifestyle that broadens my horizons. Each trip is a new adventure, filled with sandy shores, diverse flavors, and unforgettable encounters.
"""

mem0_client.add(text, user_id=USER_ID)

{'message': 'ok'}

## Fetch added memories

Fetch the added memories using the Mem0 API.

In [83]:
memories = mem0_client.get_all(user_id=USER_ID)
memories = [m["memory"] for m in memories][0].rsplit(". ")
print("User Preferences:")
print("-----------------")
for m in memories:
    print(f"- {m}")

User Preferences:
-----------------
- Is a passionate traveler with a well-stamped passport
- Prefers beaches and always chooses Airbnbs for that local touch
- Loves exploring new cultures and cuisines but doesn't like sushi
- Enjoys discovering hidden gems and connecting with locals while traveling
- Considers travel a lifestyle that broadens horizons
- Each trip is a new adventure filled with sandy shores, diverse flavors, and unforgettable encounters.


## Step 3: Create Personal Travel AI Agent

Create a personalized travel AI agent that can help you plan your trips and provide recommendations based on your preferences.

MultiOn APIs uses the extracted preferences and memories about the user to provide personalized recommendations.

In [90]:
from IPython.display import display, HTML


def print_colored(text, color='black', bold=False):
    weight = 'bold' if bold else 'normal'
    display(HTML(f"<pre style='color: {color}; font-weight: {weight};'>{text}</pre>"))

def print_separator(char='-', length=50, color='gray'):
    print_colored(char * length, color)


class TravelAIAgent:
    def __init__(self, use_memory: bool = True):
        self.use_memory = use_memory
        self.memory = MemoryClient(api_key=MEM0_API_KEY)
        self.multion = MultiOn(api_key=MULTION_API_KEY)

    def ask_question(self, question: str, user_id: str) -> str:
        relevant_memories = self._get_relevant_memories(question, user_id) if self.use_memory else ""
        prompt = self._create_prompt(question, relevant_memories)
        browse_result = self.multion.browse(cmd=prompt)
        answer = browse_result.message

        self._store_interaction(prompt, answer, user_id)
        return answer

    def _get_relevant_memories(self, query: str, user_id: str) -> str:
        memories = self.memory.search(query, user_id=user_id)
        return '\n'.join(mem['memory'] for mem in memories)

    def _create_prompt(self, question: str, relevant_memories: str) -> str:
        base_prompt = "Perform action related to the user query and preferences for travel planning:"

        if relevant_memories:
            return f"{base_prompt}\nQuestion: {question}\nMy preferences: {relevant_memories}"
        else:
            base_prompt = "Perform action related to the user query for travel planning:"
            return f"{base_prompt}\nQuestion: {question}"

    def _store_interaction(self, prompt: str, answer: str, user_id: str) -> None:
        messages = [
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": answer}
        ]
        self.memory.add(messages, user_id=user_id)

## Step 4: Run and compare performance of Travel AI Assistant (Memory vs. No Memory)

Compare responses from our Travel AI Assistant with and without memory capability.

- Enter travel questions when prompted
- See side-by-side answers from both versions
- Differences are automatically highlighted
- Type 'q' or 'exit' to quit

Run the cell below to start.

In [91]:
ai_assistant_with_memory = TravelAIAgent(use_memory=True)
ai_assistant_without_memory = TravelAIAgent(use_memory=False)

print_colored("Welcome to the Travel AI Assistant Comparison!", color='blue', bold=True)
print_colored("Ask your travel-related questions to compare responses with and without memory.", color='green')
print_colored("Type 'q' or 'exit' to quit.", color='green')

while True:
    print_separator('=', color='blue')
    question = input("Question: ")

    if question.lower() in ['q', 'exit']:
        break

    print_separator('-', color='gray')
    print_colored(f"User query: {question}", color='purple', bold=True)
    print_separator('-', color='gray')

    # Get answers from both assistants
    answer_with_memory = ai_assistant_with_memory.ask_question(question, user_id=USER_ID)
    answer_without_memory = ai_assistant_without_memory.ask_question(question, user_id=USER_ID)

    # Display answers
    print_colored("Answer (With Memory):", color='green', bold=True)
    print_colored(answer_with_memory, color='green')

    print_separator('-', color='gray')

    print_colored("Answer (Without Memory):", color='red', bold=True)
    print_colored(answer_without_memory, color='red')

    print_separator('=', color='blue')